In [1]:
# --- CARGA Y PREP ---
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import norm, qmc
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
# from sklearn.ensemble import ExtraTreesRegressor  # <- alternativo robusto

# Paths
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "initial_data" / "function_4"
assert DATA_DIR.exists(), f"No se encontró {DATA_DIR}"
# Datos
X = np.load(DATA_DIR / "initial_inputs.npy", allow_pickle=True)
y = np.load(DATA_DIR / "initial_outputs.npy", allow_pickle=True).ravel()
X = np.asarray(X, float); 
y = np.asarray(y, float).ravel()
print("Loaded from:", DATA_DIR)
print("X shape:", X.shape, " | y shape:", y.shape)
print("First row of X:", X[0])
# Semana 1
x_w1 = np.array([
0.44739007,0.4067464,0.37675635,0.39856218], dtype=float)
y_w1 = float(-0.0784261573095999) 

x_w2 = np.array([0.400828, 0.42327 , 0.353541, 0.436252], dtype=float)
y_w2 = float(0.48178123910921444)
# Guardar los previos y actualizar
X_prev, y_prev = X.copy(), y.copy()
X = np.vstack([X, x_w1, x_w2])
y = np.append(y, [y_w1, y_w2])
n, d = X.shape
print("Shapes:", X.shape, y.shape)
print("Week 1 added ->", x_w1, y_w1)
print("Week 1 added ->", x_w2, y_w2)
# Bounds desde datos + 10% acolchado, clamp a [0,1]
lo, hi = X.min(axis=0), X.max(axis=0)
pad = 0.10
lb = np.clip(lo - pad*(hi - lo), 0.0, 1.0)
ub = np.clip(hi + pad*(hi - lo), 0.0, 1.0)
print("Bounds usados:\n lb:", lb, "\n ub:", ub)
# --- ESCALADO (solo X, una sola vez) ---
sx = StandardScaler().fit(X)
Xz = sx.transform(X)

# --- GP ROBUSTO (F4: ruido alto) ---
from sklearn.gaussian_process.kernels import ConstantKernel as C
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=np.ones(d)*0.6, nu=1.5,
                                      length_scale_bounds=(0.05, 2.5)) \
         + WhiteKernel(noise_level=1e-3, noise_level_bounds=(1e-4, 1.0))
gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True,
                              n_restarts_optimizer=18, alpha=1e-6, random_state=0)
gp.fit(Xz, y)
print("Kernel final:", gp.kernel_)

y_best = y.max()

# --- Adquisiciones ---
def acq_ucb(mu, sigma, kappa=1.6): return mu + kappa * sigma
def acq_ei(mu, sigma, y_best, xi=0.03):
    improve = mu - y_best - xi
    Z = improve / (sigma + 1e-12)
    return improve * norm.cdf(Z) + sigma * norm.pdf(Z)
def acq_var(sigma): return sigma

def predict_on(gp, C, sx):
    Cz = sx.transform(C)
    mu, std = gp.predict(Cz, return_std=True)
    return mu, np.clip(std, 1e-12, None)

def too_close_Linf(c, X, tol=0.03):
    return np.any(np.max(np.abs(X - c), axis=1) < tol)

# --- CANDIDATOS: 80% TR + 20% Global ---
best_idx = np.argmax(y)
anchor = X[best_idx]
L = 0.35  # TR moderada
lb_tr = np.clip(anchor - 0.5*L*(ub - lb), 0, 1)
ub_tr = np.clip(anchor + 0.5*L*(ub - lb), 0, 1)

def lhs(lb, ub, n, seed):
    U = qmc.LatinHypercube(d=len(lb), seed=seed).random(n); return lb + U*(ub - lb)

C_tr = lhs(lb_tr, ub_tr, 50000, seed=4041)
C_gl = lhs(lb,    ub,    12000, seed=4042)
C = np.vstack([C_tr, C_gl])

# Anti-borde
edge_eps = 1e-3
mask_edges = np.all((C > edge_eps) & (C < 1 - edge_eps), axis=1)
C = C[mask_edges]

# --- PREDICCIONES + EI(xi=0.03) ---
muC, stdC = predict_on(gp, C, sx)
ei = acq_ei(muC, stdC, y_best=y_best, xi=0.03)

# --- SELECCIÓN Top-k evitando duplicados ---
order = np.argsort(-ei)
BATCH = 5
S = []
for idx in order:
    c = C[idx]
    if not too_close_Linf(c, X, tol=0.03):
        S.append(c)
        if len(S) == BATCH: break
S = np.array(S)

print(f"Sugerencias Top-{BATCH} (EI xi=0.03):\n", S)
out_path = BASE_DIR / "suggestions_f4_ei_w3.csv"
pd.DataFrame(S, columns=[f"x{i+1}" for i in range(d)]).to_csv(out_path, index=False)
print("Guardado:", out_path)

top1 = S[0]
print("Submission format (Top-1):")
print(" - ".join(f"{v:.6f}" for v in top1))

Loaded from: /Users/marvelo/Documents/ImperialCollege/capstone/initial_data/function_4
X shape: (30, 4)  | y shape: (30,)
First row of X: [0.89698105 0.72562797 0.17540431 0.70169437]
Shapes: (32, 4) (32,)
Week 1 added -> [0.44739007 0.4067464  0.37675635 0.39856218] -0.0784261573095999
Week 1 added -> [0.400828 0.42327  0.353541 0.436252] 0.48178123910921444
Bounds usados:
 lb: [0. 0. 0. 0.] 
 ub: [1. 1. 1. 1.]
Kernel final: 1.01**2 * Matern(length_scale=[2.5, 2.06, 2.5, 2.5], nu=1.5) + WhiteKernel(noise_level=0.0001)
Sugerencias Top-5 (EI xi=0.03):
 [[0.39992257 0.4814963  0.41761449 0.45510322]
 [0.37889415 0.47751338 0.428625   0.44675381]
 [0.40491767 0.48355283 0.40822442 0.44729766]
 [0.38883959 0.48119698 0.41954564 0.43277532]
 [0.39303284 0.47466474 0.43034456 0.44328637]]
Guardado: /Users/marvelo/Documents/ImperialCollege/capstone/suggestions_f4_ei_w3.csv
Submission format (Top-1):
0.399923 - 0.481496 - 0.417614 - 0.455103


/opt/anaconda3/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 2.5. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 2 of parameter k1__k2__length_scale is close to the specified upper bound 2.5. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 3 of parameter k1__k2__length_scale is close to the specified upper bound 2.5. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py

In [3]:
import numpy as np

cand = np.array([0.39992257, 0.4814963,  0.41761449, 0.45510322])
def too_close_Linf(c, X, tol=0.02):
    return np.any(np.max(np.abs(X - c), axis=1) < tol)

print("Within [0,1]? ", np.all((cand >= 0) & (cand <= 1)))
print("Too close to existing (L_inf<0.02)? ", too_close_Linf(cand, X, tol=0.02))
print("Submission:\n" + " - ".join(f"{v:.6f}" for v in cand))

Within [0,1]?  True
Too close to existing (L_inf<0.02)?  False
Submission:
0.399923 - 0.481496 - 0.417614 - 0.455103


Bitácora (2–3 líneas para F4)
	•	Method: GP (Matern ν=1.5, normalize_y=True) + EI (ξ=0.07), trust region alrededor de W1, anti-borde y anti-duplicado.
	•	Rationale: El Top-5 EI se concentró cerca del punto de la semana 1; elegí el Top-1 para refinar localmente en una región prometedora pese al ruido.
	•	Submission: 0.400828 - 0.423270 - 0.353541 - 0.436252.


🧾 Function 4 – Week 3 Log

Update:
This week, I added the second data point from Week 2, which showed a clear improvement over Week 1 (from –0.078 to +0.48). The dataset now includes 32 samples.

Model setup:
I kept a Gaussian Process with a Matern (ν = 1.5) kernel and added a ConstantKernel multiplier to better handle varying output scales. The model used normalize_y=True and WhiteKernel noise regularisation. The fitted kernel

In [ ]:
1.01² × Matern(length_scale=[2.5, 2.06, 2.5, 2.5]) + WhiteKernel(noise_level=0.0001)

indicates a smooth, low-noise surface—ideal for refining near the current optimum.

Acquisition strategy:
I applied Expected Improvement (EI) with a lower exploration parameter (ξ = 0.03), combining 80 % sampling inside a trust region centred on the best previous point and 20 % global candidates for robustness.
This balances local exploitation with mild exploration.

Result:
The suggested next query was: 0.399923 – 0.481496 – 0.417614 – 0.455103

which lies close to the prior high-performing region but shifts slightly along x₂ and x₃ to probe a nearby ridge of potential improvement.

Next steps:
If this new evaluation confirms further gain, I’ll narrow ξ to 0.02 and maintain the same kernel.
If performance stagnates, I’ll re-expand the trust region and re-tune noise bounds to encourage broader exploration.